# Sistema de recomendación con KD Trees

Realizamos un sistema de recomendación basado en la estrutura de datos KD Tree en un notebook interactivo de Jupyter utilizando la librería iPyWidgets y scikit-learn.

*Instalamos las librerías*

In [ ]:
%pip install ipywidgets

In [ ]:
%pip install pandas

In [ ]:
%pip install scikit-learn

*Importaciones*

In [95]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import random

## ¿En qué consiste nuestro notebook?

Tendrás la posibilidad de introducir diferentes tipos de prendas, tales como camisas, camisetas, jeans o faldas, junto con sus respectivas características como precio, material y el público al que van dirigidas. Además, podrás generar conjuntos de datos aleatorios que representen diversas prendas.

El propósito principal de este notebook es entrenar un modelo basado en KD Trees (árboles KD) utilizando los datos de las prendas. Este modelo se emplea para determinar la recomendación de un producto. Es decir, a partir de las características proporcionadas de una prenda específica, el modelo puede evaluar si es recomendable o no para cierto tipo de público o situación.

### Agrega prendas a mano al dataset de entrenamiento

In [88]:
data = {
    'Tipo de Prenda': [],
    'Precio': [],
    'Material': [],
    'Publico': [],
    'Recomendado': []  
}

title = widgets.Label(value='Formulario de Registro de Prendas', style={'font-weight': 'bold', 'font-size': '20px'})

type = widgets.Dropdown(options=['Camisa', 'Camiseta', 'Jean', 'Falda'], description='Tipo de Prenda:')
price = widgets.FloatSlider(value=75000, min=15000, max=150000, step=100, description='Precio:')
material = widgets.Dropdown(options=['Algodón', 'Seda', 'Cuero', 'Lana'], description='Material:')
public = widgets.Dropdown(options=['Infantil', 'Adulto'], description='Publico:')
recommended = widgets.Dropdown(options=['Sí', 'No'], description='Recomendado')
add_button = widgets.Button(description='Agregar Prenda')

def add_product(sender):
    data['Tipo de Prenda'].append(type.value)
    data['Precio'].append(price.value)
    data['Material'].append(material.value)
    data['Publico'].append(public.value)
    data['Recomendado'].append(1 if recommended.value == 'Sí' else 0)
    print(f'Prenda agregada: {type.value}, {price.value}, {material.value}, {public.value}, {recommended.value}')

add_button.on_click(add_product)

display(title, type, price, material, public, recommended, add_button)

Label(value='Formulario de Registro de Prendas')

Dropdown(description='Tipo de Prenda:', options=('Camisa', 'Camiseta', 'Jean', 'Falda'), value='Camisa')

FloatSlider(value=75000.0, description='Precio:', max=150000.0, min=15000.0, step=100.0)

Dropdown(description='Material:', options=('Algodón', 'Seda', 'Cuero', 'Lana'), value='Algodón')

Dropdown(description='Publico:', options=('Infantil', 'Adulto'), value='Infantil')

Dropdown(description='Recomendado', options=('Sí', 'No'), value='Sí')

Button(description='Agregar Prenda', style=ButtonStyle())

Prenda agregada: Camisa, 75000.0, Algodón, Infantil, Sí
Prenda agregada: Jean, 75000.0, Algodón, Infantil, Sí
Prenda agregada: Jean, 75000.0, Algodón, Adulto, Sí
Prenda agregada: Jean, 75000.0, Seda, Adulto, No
Prenda agregada: Jean, 75000.0, Lana, Adulto, No
Prenda agregada: Jean, 75000.0, Lana, Infantil, No
Prenda agregada: Camiseta, 75000.0, Lana, Infantil, No
Prenda agregada: Camiseta, 75000.0, Lana, Adulto, Sí
Prenda agregada: Falda, 75000.0, Lana, Infantil, Sí
Prenda agregada: Falda, 75000.0, Lana, Adulto, No
Prenda agregada: Falda, 75000.0, Cuero, Adulto, Sí
Prenda agregada: Falda, 75000.0, Cuero, Infantil, No
Prenda agregada: Jean, 119100.0, Cuero, Infantil, No
Prenda agregada: Jean, 38400.0, Cuero, Infantil, No
Prenda agregada: Jean, 38400.0, Cuero, Infantil, Sí
Prenda agregada: Jean, 86400.0, Cuero, Adulto, Sí


### O genera prendas aleatorias para el dataset de entrenamiento

In [100]:
data = {
    'Tipo de Prenda': [],
    'Precio': [],
    'Material': [],
    'Publico': [],
    'Recomendado': []  
}

tipos_prenda = ['Camisa', 'Camiseta', 'Jean', 'Falda']
materiales = ['Algodón', 'Seda', 'Cuero', 'Lana']
publicos = ['Infantil', 'Adulto']
opciones_recomendado = ['Sí', 'No']

num_datos = 100 

for _ in range(num_datos):
    tipo_prenda = random.choice(tipos_prenda)
    precio = random.randint(15000, 150000)
    material_prenda = random.choice(materiales)
    publico_prenda = random.choice(publicos)
    recomendado = random.choice(opciones_recomendado)
    
    data['Tipo de Prenda'].append(tipo_prenda)
    data['Precio'].append(precio)
    data['Material'].append(material_prenda)
    data['Publico'].append(publico_prenda)
    data['Recomendado'].append(1 if recomendado == 'Sí' else 0)

### Visualiza los productos agregados en el dataset que utilizaremos para entrenar el modelo

In [101]:
products = pd.DataFrame(data)
products

,Tipo de Prenda,Precio,Material,Publico,Recomendado
0,Camiseta,121737,Algodón,Infantil,0
1,Camisa,37233,Lana,Adulto,0
2,Jean,129098,Lana,Adulto,1
3,Camiseta,31614,Algodón,Adulto,1
4,Camiseta,114787,Cuero,Adulto,0
...,...,...,...,...,...
95,Falda,49591,Algodón,Adulto,1
96,Falda,135334,Cuero,Infantil,0
97,Falda,76682,Algodón,Infantil,1
98,Jean,72467,Lana,Infantil,1


### Entrena al modelo y define el número de vecinos a considerar para la búsqueda de los N vecinos más cercanos

In [102]:
X = products[['Tipo de Prenda', 'Precio', 'Material', 'Publico']]
y = products['Recomendado']

X = pd.get_dummies(X)

knn = KNeighborsClassifier(n_neighbors=3)  # Número de vecinos a considerar
knn.fit(X, y)

KNeighborsClassifier(n_neighbors=3)

### ¡Haz predicciones!

In [103]:
title = widgets.Label(value='Realiza una predicción', style={'font-weight': 'bold', 'font-size': '20px'})

@interact(Type=['Camisa', 'Camiseta', 'Jean', 'Falda'],
          Material=['Algodón', 'Seda', 'Cuero', 'Lana'],
          Price=(15000, 150000),
          Public=['Infantil', 'Adulto'])
def predict_recommended(Price=75000, Type='Camisa', Material='Algodón', Public='Infantil'):
    test_data = pd.DataFrame({'Tipo de Prenda': [Type], 'Precio': [Price], 'Material': [Material], 'Publico': [Public]})
    test_data_encoded = pd.get_dummies(test_data)
    missing_cols = set(X.columns) - set(test_data_encoded.columns)
    for c in missing_cols:
        test_data_encoded[c] = 0
        
    test_data_encoded = test_data_encoded[X.columns]
    
    
    prediction = knn.predict(test_data_encoded)
    if prediction[0] == 1:
        print("¡El producto es recomendado!")
    else:
        print("El producto no es recomendado.")

interactive(children=(IntSlider(value=75000, description='Price', max=150000, min=15000), Dropdown(description…